In [3]:
def load_yolo_labels(label_path):
    """
    Load YOLO format labels from the given path.

    Parameters:
    label_path: str
        Path to the label file.

    Returns:
    list of tuples
        List of bounding boxes (class_id, center_x, center_y, width, height).
    """
    with open(label_path, 'r') as file:
        labels = []
        for line in file:
            class_id, center_x, center_y, width, height = map(float, line.strip().split())
            labels.append((int(class_id), center_x, center_y, width, height))
        return labels

In [11]:
from PIL import Image
import cv2
import numpy as np

def annotate_image(img, yolo_bbox):
    """
    Annotate the image with the bounding boxes of the detected objects.

    Parameters:
    img: PIL.Image
        The image to annotate.
    yolo_bbox: list of tuples
        List of YOLO format bounding boxes (class_id, center_x, center_y, width, height).

    Returns:
    PIL.Image
        The annotated image.
    """
    img = np.array(img)
    height, width, _ = img.shape
    
    for bbox in yolo_bbox:
        class_id, center_x, center_y, bbox_width, bbox_height = bbox
        
        # Convert YOLO format to corner coordinates
        x1 = int((center_x - bbox_width / 2) * width)
        y1 = int((center_y - bbox_height / 2) * height)
        x2 = int((center_x + bbox_width / 2) * width)
        y2 = int((center_y + bbox_height / 2) * height)
        
        # Draw rectangle around detected object
        color = (0, 255, 0)  # Green color for bounding box
        thickness = 2  # Thickness of bounding box lines
        img = cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
        
        # Optionally, add the class ID or label
        if class_id == 0: 
            state = "[CLOSED]"
        elif class_id == 1: 
            state = "[SEMI-OPEN]"
        elif class_id == 2: 
            state = "[OPEN]"
        else:
            return Image.fromarray(img)
        label = f"Refuelling Port {state}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        label_x = x1
        label_y = y1 - 10 if y1 - 10 > 10 else y1 + 10
        cv2.rectangle(img, (label_x, label_y - label_size[1]), 
                      (label_x + label_size[0], label_y + label_size[1]), color, -1)
        cv2.putText(img, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    return Image.fromarray(img)


### Generate a images grid 

In [47]:
import random
import os
from PIL import Image

from torchvision.utils import make_grid, save_image
import torchvision.transforms.functional as TF

folder_path = "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/close/train/all"
grid_size = 3

images_folder_path = folder_path + "/images"
labels_folder_path = folder_path + "/labels"

images = []
for i in range(grid_size * grid_size):
    image_file = random.choice(os.listdir(images_folder_path))
    image_path = os.path.join(images_folder_path, image_file)
    label_path = os.path.join(labels_folder_path, os.path.splitext(image_file)[0] + '.txt')
    
    image = Image.open(image_path)
    labels = load_yolo_labels(label_path)
    annotated_image = annotate_image(image, labels)
    
    # Transform the annotated image to tensor
    image_tensor = TF.to_tensor(annotated_image)
    images.append(image_tensor)

# Create a grid of images
grid_image = make_grid(images, nrow=grid_size)
save_image(grid_image, "grid_closed_image2.png")